# Ejercicio 1: Redes de cooperación en cursos

Vamos a trabajar con datos del proyecto "Juecoo" realizado por investigadores de nuestra universidad. Se le entrega un subconjunto anonimizado con una selección de datosdatos disponible en la carpeta. El grupo 1 trabaje con el curso 1 y el grupo 2 con el curso 2.

1. Identifique las variables disponibles con las que cuente y realice una breve estadística descriptiva.

Cargamos los dato, contamos con dos tablas: data de diadas y data de individuos.

In [ ]:
diadas <- read.csv("./data_cursos_juecoo/Data_diadas_ayudantia4_ej1.csv", header=T, sep = ',')
individuos <- read.csv("./data_cursos_juecoo/Data_indiv_ayudantia4_ej1.csv", header=T, sep = ',')

Le podemos dar un vistazo general a cada data set:

In [ ]:
glimpse(diadas)

In [ ]:
glimpse(individuos)

Podemos usar skimr para hacer una visión general de ambos data frames:

In [ ]:
# Cargar skimr
library("skimr")

# Usamos skim() para inspeccionar los datos
skim(individuos)

In [ ]:
# Usamos skim() para inspeccionar los datos
skim(diadas)

2. Filtre sus datos para el curso con el cual le toca trabajar y revise estadísticas de dicho curso.

In [ ]:
diadas_filtrado <- diadas %>% filter(Id_Curso==11)
individuos_filtrado <- individuos %>% filter(Id_Curso==11)

In [ ]:
skim(individuos_filtrado)

skim(diadas_filtrado)

3. Creemos una red en base a los datos, en los cuales definiremos un link entre dos individuos si hay al menos un envio de 10 entre ellos. No importa quien realice el envio.  Esta es una red no dirigida sin pesos. 

In [ ]:
edgelist_1 <- diadas_filtrado %>% 
              filter(weight==10) %>%
              select(Id_Emisor, Id_Receptor) %>%
              filter(!duplicated(paste0(pmax(Id_Emisor, Id_Receptor), pmin(Id_Emisor, Id_Emisor)))) #removemos duplicados, porque queremos un link entre cada par


view(edgelist_1)


creamos el objeto de igraph

In [ ]:
g1 <- igraph::graph_from_data_frame(edgelist_1 , directed = F) 

4. Realice una visualización de la red.


In [ ]:
plot(g1)

Vemos que hay más de un link, porque no borramos los duplicados. Podemos usar simplfy para esto:

In [ ]:
g1 <- simplify( g1, remove.multiple = T, remove.loops = F, 
                 edge.attr.comb=list(weight="sum", type="ignore") )


In [ ]:
plot(g1, vertex.label.dist=1.5)

Ahora tenemos un plot más limpio.



5. Calcule la centralidad de grado y reagile un histograma de esta. ¿QUé información puede interpretar de este resultado.

6. Ahora creemos una segunda red para el curso, pero esta vez definiremos un link entre dos individuos hay un envio de 10 entre ellos, pero preservaremos la información de quien realiza el envio. Es decir, debe crear una edgelist con dos elementos: from y to. Esta es una red no dirigida sin pesos. 


In [ ]:
edgelist_2 <- diadas_filtrado %>% 
              filter(weight==10) %>%
              select(Id_Emisor, Id_Receptor)

view(edgelist_2)

creamos el objeto de igraph

In [ ]:
g2 <- igraph::graph_from_data_frame(edgelist_2 , directed = T) 

In [ ]:
plot(g2)

Podemos ver las propiedades de nuestros grafos, simplemente observado el objeto:

In [ ]:
g2

7. Calcule la centralidad de grado, in degree y outdegree y realice un histograma de estas. ¿Qué información puede interpretar de este resultado.

In [ ]:
degree_g2 <- degree(g2)
indegree_g2 <- degree(g2, mode="in")
outdegree_g2 <- degree(g2, mode="out")


Haciendo más lindo el grafo:


In [ ]:
#tamaño de la flecha
E(g2)$arrow.size <- .2

#Tamaño es por indegree del nodo

V(g2)$size <- V(g2)$indegree_g2

plot(g2)


8. Agreguege a ambas visualizaciones el grado como atributo del nodo y úselo para su tamaño.

Calculamos los grados:

In [ ]:
degree_g2 <- degree(g2)
indegree_g2 <- degree(g2, mode="in")
outdegree_g2 <- degree(g2, mode="out")

indegree_g2

Si bien, el orden de los nodos es el de la base datos. Siempre es bueno saber el id tb:

In [ ]:
id <- rownames(indegree_g2)
indegree_g2 <- cbind(id, indegree_g2)

indegree_g2

podemos agregar este objeto como atributo al grafo también:

g4 <- set_graph_attr(g4, "something", "A thing")

In [ ]:
igraph::set_graph_attr(g2,"indegree_g2", indegree_g2 )

In [ ]:
vertex_attr(g2)

9. Agregue a ambas visualizaciones el género como un atributo del nodo y coloree los nodos de acuerdo a este. 


Lo que tenemos que hacer ahora es agregar una info extra de los nodos, para eso volveremos a crear el objeto de grafo 

In [ ]:
#Lista de nodos con atributos de nodo

nodos <- individuos_filtrado %>% select(Id, Sexo)

nodos

Y al crear el objeto de red, le indicamos de donde salen los vertices y de donde los nodos:


In [ ]:
g3 <- igraph::graph_from_data_frame(edgelist_2, vertices=nodos, directed = T)

Si ahora inspeccionamos el objeto, toda la info de ese data frame queda como atributo de los nodos.

In [ ]:
g3

In [ ]:
vertex_attr(g3)

Entonces, ahora podemos colorear los nodos con esa info:

In [ ]:
#tamaño de la flecha
E(g3)$arrow.size <- .2

#tamaño del nodo es el grado
V(g3)$size <- degree_g2
#V(g3)$size <- V(g3)$degree_g2 #cuando se lo agregamos como atributo

#Tamaño es por indegree del nodo

V(g3)$color <- case_when( V(g3)$Sexo=="F" ~ "pink",  #"#2F91A6" , # Female un color
                          V(g3)$Sexo=="M" ~ "#A3D3D2D9"  ) # Male otro color

In [ ]:
plot(g3)

(Propuesto: Finalmente, cree una lista de adjaciencia con pesos para su curso. Esta lista debe tener columnas elementos: from, to, weight (que corresponde al link) y replique la visualización, esta vez agregando un atributo al link que indique el grosor como el peso)